In [1]:
%reload_ext autoreload
%autoreload 2

import myforexclass as mf

In [2]:
test = mf.forex_backtest_class(["ETH-USD"],"2024-06-01" , "2024-07-01", "1d" , 0.0001 , 100000)

[*********************100%%**********************]  1 of 1 completed

Data of ETH-USD downloded.


In [ ]:
test.data

In [ ]:
test.plot_data()

In [40]:
test.sma("ETH-USD" , 37,50)

0.81102

In [41]:
test.temp_data

,Close,returns,sma_s,sma_l,pos,trades,str_sma,str_net,cum_str_net
Date,,,,,,,,,
2023-02-21,1658.036377,-0.026567,1611.043358,1543.403093,1,0.0,-0.026567,-0.026567,0.973783
2023-02-22,1643.231689,-0.008969,1612.837904,1551.972151,1,0.0,-0.008969,-0.008969,0.965088
2023-02-23,1651.073853,0.004761,1615.087303,1559.863096,1,0.0,0.004761,0.004761,0.969694
2023-02-24,1608.373169,-0.026203,1617.597201,1567.021787,1,0.0,-0.026203,-0.026203,0.944615
2023-02-25,1594.914917,-0.008403,1618.742023,1573.532505,1,0.0,-0.008403,-0.008403,0.936711
...,...,...,...,...,...,...,...,...,...
2024-06-26,3369.477295,-0.007555,3648.440608,3490.350937,1,0.0,-0.007555,-0.007555,0.796043
2024-06-27,3444.800537,0.022108,3639.129467,3499.773804,1,0.0,0.022108,0.022108,0.813838
2024-06-28,3373.635986,-0.020875,3629.302932,3506.526113,1,0.0,-0.020875,-0.020875,0.797026


In [45]:
t=test.sma_backtest("ETH-USD",37,50)

Testing SMA Strategy | ETH-USD | SMA_S= 37 | SMA_L= 50
---------------------------------------------------------------------------
Initial amount is : 100000
2023-02-20 Buying 58 for 1702.67517 with 0.0001 spread. Net price is 1702.67522 and current balance is 1244.84
2023-03-10 Selling 58 for 1429.15808 with 0.0001 spread. Net price is 1429.15803 and current balance is 84136.01
** Closed last long position !
2023-03-10 Selling 58 for 1429.15808 with 0.0001 spread. Net price is 1429.15803 and current balance is 167027.18
2023-03-19 Buying 58 for 1785.57654 with 0.0001 spread. Net price is 1785.57659 and current balance is 63463.74
** Closed last short position !
2023-03-19 Buying 35 for 1785.57654 with 0.0001 spread. Net price is 1785.57659 and current balance is 968.56
2023-05-21 Selling 35 for 1804.53137 with 0.0001 spread. Net price is 1804.53132 and current balance is 64127.16
** Closed last long position !
2023-05-21 Selling 35 for 1804.53137 with 0.0001 spread. Net price is 1804.

In [43]:
test.count_months()

(                  Close   returns        sma_s        sma_l  pos  trades  \
 Date                                                                       
 2023-02-21  1658.036377 -0.026567  1611.043358  1543.403093    1     0.0   
 2023-02-22  1643.231689 -0.008969  1612.837904  1551.972151    1     0.0   
 2023-02-23  1651.073853  0.004761  1615.087303  1559.863096    1     0.0   
 2023-02-24  1608.373169 -0.026203  1617.597201  1567.021787    1     0.0   
 2023-02-25  1594.914917 -0.008403  1618.742023  1573.532505    1     0.0   
 ...                 ...       ...          ...          ...  ...     ...   
 2024-06-26  3369.477295 -0.007555  3648.440608  3490.350937    1     0.0   
 2024-06-27  3444.800537  0.022108  3639.129467  3499.773804    1     0.0   
 2024-06-28  3373.635986 -0.020875  3629.302932  3506.526113    1     0.0   
 2024-06-29  3372.967773 -0.000198  3618.385109  3515.789644    1     0.0   
 2024-06-30  3432.889160  0.017609  3610.437935  3526.215386    1     0.0   

In [ ]:
test.best_param_ema("ETH-USD")

In [ ]:
t=test.ema_backtest("ETH-USD",9,87)

In [ ]:
test.best_param_dema("ETH-USD")

In [ ]:
t=test.dema_backtest("ETH-USD",31,54)

In [4]:
test.best_param_rsi("ETH-USD")

ValueError: attempt to get argmax of an empty sequence

In [ ]:
t=test.rsi_backtest("ETH-USD",17,25,71)

In [ ]:
test.best_macd("ETH-USD")

In [ ]:
t=test.macd_backtest("ETH-USD",19,39,14)

In [ ]:
test.best_bollinger("ETH-USD")

In [ ]:
t=test.bollinger_backtest("ETH-USD",44,3)

In [ ]:
test.best_stochastic("ETH-USD")

In [ ]:
t=test.stochastic_backtest("ETH-USD",14,2)

In [ ]:
test.atr("ETH-USD",14)

In [ ]:
test.adx("ETH-USD",14)

In [ ]:
test.obv("ETH-USD")

In [ ]:
t=test.ichimoku("ETH-USD")
print(t[0].tail(50))
print(t[1])

In [ ]:
test.ichimoku_backtest("ETH-USD")

In [7]:
test = mf.forex_backtest_class(["EURUSD=X"],"2024-01-01" , "2024-07-01", "1d" , 0.0001 , 100000)
test.data

[*********************100%%**********************]  1 of 1 completed

Data of EURUSD=X downloded.
Testing Ichimuko Strategy | EURUSD=X | s= 14 | m= 26 | l=52
---------------------------------------------------------------------------
Initial amount is : 100000
2024-01-15 Selling 91364 for 1.09457 with 0.0001 spread. Net price is 1.09452 and current balance is 199999.73
2024-03-14 Buying 91364 for 1.09553 with 0.0001 spread. Net price is 1.09558 and current balance is 99903.16
** Closed last short position !
2024-03-14 Buying 91187 for 1.09553 with 0.0001 spread. Net price is 1.09558 and current balance is 0.51
2024-04-03 Selling 91187 for 1.07686 with 0.0001 spread. Net price is 1.07681 and current balance is 98191.58
** Closed last long position !
2024-04-03 Selling 91187 for 1.07686 with 0.0001 spread. Net price is 1.07681 and current balance is 196382.65
2024-05-21 Buying 91187 for 1.08608 with 0.0001 spread. Net price is 1.08613 and current balance is 97341.71
** Closed last short position !
2024-05-21 Buying 89622 for 1.08608 with 0.0001 spread. Net

(-4.02, 10, -3.14, 95978.57)

In [ ]:
test.ichimoku_backtest("EURUSD=X")

In [ ]:
import MetaTrader5 as mt5
import myforexclass as mf
import datetime

start_year = 2024
start_month = 3
end_year = 2024
end_month = 6
interval = "1h"
tickers=["ETH-USD"]
result_data = mf.pd.DataFrame(columns=["date","ticker","return_hold","short_sma","long_sma","return_sma","short_ema","long_ema","return_ema",
                                       "short_dema","long_dema","return_dema","period_rsi","m_down_rsi","m_up_rsi"])

# حلقه برای دریافت داده‌های هر ماه
year = start_year
month = start_month
for ticker in tickers :
    while year < end_year or (year == end_year and month <= end_month):
        start_month_date = datetime.date(year, month, 1)
        if month == 12:
            end_month_date = datetime.date(year+1, 1, 1) - datetime.timedelta(days=1)
            year += 1
            month = 1
        else:
            end_month_date = start_month_date.replace(month=month+1) - datetime.timedelta(days=1)
            month += 1

        # نمایش نوار پیشرفت
        print(f" دریافت داده‌های ماه قبل {month}-{year} نماد {ticker}")
        t = mf.forex_backtest_class([ticker],start_month_date , end_month_date , interval )
        
        #******************************** SMA
        (s_sma,l_sma)= t.best_param_sma(ticker)
        r_sma=t.sma(ticker,s_sma,l_sma)
        r_h=t.get_perf_hold(-1)
        #******************************* EMA
        (s_ema,l_ema)= t.best_param_ema(ticker)
        r_ema=t.ema(ticker,s_ema,l_ema)
        #******************************* DMA
        (s_dema,l_dema)= t.best_param_dema(ticker)
        r_dema=t.dema(ticker,s_dema,l_dema)
        #******************************* RSI
        (p_rsi,d_rsi,u_rsi)= t.best_param_rsi(ticker)
        r_rsi=t.rsi(ticker,p_rsi,d_rsi,u_rsi)
        
        
        
        
        
        
        #new_row = mf.pd.Series([end_month_date,ticker ,r_h,s_sma, l_sma, r_sma,s_ema,l_ema,r_ema,s_dema,l_dema,r_dema,p_rsi,d_rsi,u_rsi], index=result_data.columns)
        #result_data = mf.pd.concat([result_data, new_row.to_frame().T], ignore_index=True)

result_data.set_index('date', inplace=True)
print(result_data)



 دریافت داده‌های ماه قبل 4-2024 نماد ETH-USD


[*********************100%%**********************]  1 of 1 completed


Data of ETH-USD downloded.
